# This next cell is "hidden", but you WILL need to "RUN" it.  
It contains
all of the other code that you need to run the project.  You can look at it if
you want, but do NOT!!! change any of it.  If you look at it and then decide
to hide it again, select the cell and do "Collapse selected code" in the "View" menu

In [ ]:
#
# This is the main program and important parts
# you should NOT change any of this, but you SHOULD "RUN" it to evaluate all of the functions...
#
import numpy as np
import random
import sys
import math
from calysto.display import display, clear_output
import plotly.graph_objects as go


ROW_COUNT = 6
COLUMN_COUNT = 7

PLAYER = 0
AI = 1

EMPTY = 0
PLAYER_PIECE = 1
AI_PIECE = 2

WINDOW_LENGTH = 4


# https://plotly.com/python/axes/#setting-the-range-of-axes-manually
# Ostermann wrote this part - that was fun!!!
def plot_board(board, winner):

    fig_width=500
    fig_height=(0.9)*fig_width
    spotsize = 0.5*(fig_width/7)

    print_board(board)

    fig = go.Figure(layout=go.Layout(height=fig_height, width=fig_width))
    
    hx=hy=ax=ay=ex=ey=[]   # set them all to an empty list
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT):
            who = board[r][c]
            if who == 1:
                hy=hy+[r+1]
                hx=hx+[c+1]
            elif who == 2:
                ay=ay+[r+1]
                ax=ax+[c+1]
            else:
                ey=ey+[r+1]
                ex=ex+[c+1]
    
    fig.add_trace(go.Scatter(
        x=hx,
        y=hy,
        marker=dict(color="crimson", size=spotsize),
        mode="markers",
        name="Human",
    ))

    fig.add_trace(go.Scatter(
        x=ax,
        y=ay,
        marker=dict(color="gold", size=spotsize),
        mode="markers",
        name="AI",
    ))
    
    fig.add_trace(go.Scatter(
        x=ex,
        y=ey,
        marker=dict(color="black", size=spotsize),
        mode="markers",
        name="empty",
    ))

    fig.update_xaxes(range=[0.25, 7.75],gridwidth=0.5,gridcolor='rgb(0,0,128)')
    fig.update_yaxes(range=[0.25, 6.75],gridwidth=0.5,gridcolor='rgb(0,0,128)')
    fig.update_yaxes(showticklabels=False)
    fig.update_xaxes(tick0=1.0, dtick=1.0)

    if winner != "":
        fig.add_annotation(
        x=4,
        y=5,
        xref="x",
        yref="y",
        text=winner,
        font=dict(
            family="Courier New, monospace",
            size=48,
            color="red"
            ),
        align="center",
        ax=0,
        ay=0,
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="yellow",
        opacity=0.50
        )
    
    fig.update_layout(plot_bgcolor='rgb(0,0,255)')                  
            
    #clear_output(wait=True)
    fig.show()
    
    
def create_board():
    board = np.zeros((ROW_COUNT,COLUMN_COUNT))
    return board

def drop_piece(board, row, col, piece):
    board[row][col] = piece

def is_valid_location(board, col):
    return board[ROW_COUNT-1][col] == 0

def get_next_open_row(board, col):
    for r in range(ROW_COUNT):
        if board[r][col] == 0:
            return r

def print_board(board):
    print(np.flip(board, 0))

#
#  Students write this as part of the program...
#
#def winning_move(board, piece):
#    return False
 

def evaluate_window(window, piece):
    score = 0
    opp_piece = PLAYER_PIECE
    if piece == PLAYER_PIECE:
        opp_piece = AI_PIECE
        
    print(window)        

    if window.count(piece) == 4:
        score += 100
    elif window.count(piece) == 3 and window.count(EMPTY) == 1:
        score += 5
    elif window.count(piece) == 2 and window.count(EMPTY) == 2:
        score += 2

    if window.count(opp_piece) == 3 and window.count(EMPTY) == 1:
        score -= 4

    return score

def score_position(board, piece):
    score = 0

    ## Score center column
    center_array = [int(i) for i in list(board[:, COLUMN_COUNT//2])]
    center_count = center_array.count(piece)
    score += center_count * 3

    ## Score Horizontal
    for r in range(ROW_COUNT):
        row_array = [int(i) for i in list(board[r,:])]
        for c in range(COLUMN_COUNT-3):
            window = row_array[c:c+WINDOW_LENGTH]
            score += evaluate_window(window, piece)

    ## Score Vertical
    for c in range(COLUMN_COUNT):
        col_array = [int(i) for i in list(board[:,c])]
        for r in range(ROW_COUNT-3):
            window = col_array[r:r+WINDOW_LENGTH]
            score += evaluate_window(window, piece)

    ## Score positive sloped diagonal
    for r in range(ROW_COUNT-3):
        for c in range(COLUMN_COUNT-3):
            window = [board[r+i][c+i] for i in range(WINDOW_LENGTH)]
            score += evaluate_window(window, piece)
    
    ## Score positive sloped diagonal
    for r in range(ROW_COUNT-3):
        for c in range(COLUMN_COUNT-3):
            window = [board[r+3-i][c+i] for i in range(WINDOW_LENGTH)]
            score += evaluate_window(window, piece)

    return score

def is_terminal_node(board):
    return winning_move(board, PLAYER_PIECE) or \
        winning_move(board, AI_PIECE) or \
        len(get_valid_locations(board)) == 0

def minimax(board, depth, alpha, beta, maximizingPlayer):
    valid_locations = get_valid_locations(board)
    is_terminal = is_terminal_node(board)
    if depth == 0 or is_terminal:
        if is_terminal:
            if winning_move(board, AI_PIECE):
                return (None, 100000000000000)
            elif winning_move(board, PLAYER_PIECE):
                return (None, -10000000000000)
            else: # Game is over, no more valid moves
                return (None, 0)
        else: # Depth is zero
            return (None, score_position(board, AI_PIECE))
    if maximizingPlayer:
        value = -math.inf
        column = random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = board.copy()
            drop_piece(b_copy, row, col, AI_PIECE)
            new_score = minimax(b_copy, depth-1, alpha, beta, False)[1]
            if new_score > value:
                value = new_score
                column = col
            alpha = max(alpha, value)
            if alpha >= beta:
                break
        return column, value

    else: # Minimizing player
        value = math.inf
        column = random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = board.copy()
            drop_piece(b_copy, row, col, PLAYER_PIECE)
            new_score = minimax(b_copy, depth-1, alpha, beta, True)[1]
            if new_score < value:
                value = new_score
                column = col
            beta = min(beta, value)
            if alpha >= beta:
                break
        return column, value

def get_valid_locations(board):
    valid_locations = []
    for col in range(COLUMN_COUNT):
        if is_valid_location(board, col):
            valid_locations.append(col)
    return valid_locations

def pick_best_move(board, piece):
    valid_locations = get_valid_locations(board)
    best_score = -10000
    best_col = random.choice(valid_locations)
    for col in valid_locations:
        row = get_next_open_row(board, col)
        temp_board = board.copy()
        drop_piece(temp_board, row, col, piece)
        score = score_position(temp_board, piece)
        if score > best_score:
            best_score = score
            best_col = col

    return best_col

def play_game():

    board = create_board()
    plot_board(board,"")
    game_over = False

    # who plays first is random
    turn = random.randint(PLAYER, AI)

    while not game_over:

        if turn == 0:
            selection = int(input("Player 1 make your selection (1-7)"))  
            col = selection-1  # change from (1..7) to (0..6)

            if is_valid_location(board, col):
                row = get_next_open_row(board, col)
                drop_piece(board, row, col, PLAYER_PIECE)

                if winning_move(board, PLAYER_PIECE):
                    plot_board(board,"Human Player Wins!!")
                    print("\n\nHuman Player Wins!!\n\n")
                    game_over = True
                else:
                    turn = (turn+1) % 2
                    plot_board(board,"")

        if turn == AI and not game_over:
            col, minimax_score = minimax(board, 5, -math.inf, math.inf, True)

            if is_valid_location(board, col):
                row = get_next_open_row(board, col)
                drop_piece(board, row, col, AI_PIECE)

                if winning_move(board, AI_PIECE):
                    plot_board(board,"AI Player Wins!!")
                    print("\n\nAI Player Wins\n\n")
                    game_over = True
                else:
                    turn = (turn+1) % 2
                    plot_board(board,"")

## These are the routines that you need to write

In [21]:
#
# These routines will tell the AI if the board position passed is a "winning" board for player "piece"
#
#
def winning_move_horizontal(board, piece):
    # Check horizontal locations for win by 'piece'
    # return True if piece wins the board by having 4 in a row horizontally
    # anywhere on the board
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT):
            if "WRITE THIS CODE - 4 in a row":
                return True
            
def winning_move_vertical(board, piece):
    # Check vertical locations for win by 'piece'
    # return True if piece wins the board by having 4 in a row vertically
    # anywhere on the board
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT):
            if "WRITE THIS CODE - 4 in a row":
                return True
            
def winning_move_diagonal_up(board, piece):
    # Check diagonal locations for win by 'piece'
    # return True if piece wins the board by having 4 in a row diagonally (going up to the right)
    # anywhere on the board
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT):
            if "WRITE THIS CODE - 4 in a row":
                return True

def winning_move_diagonal_down(board, piece):
    # Check diagonal locations for win by 'piece'
    # return True if piece wins the board by having 4 in a row diagonally (going down to the right)
    # anywhere on the board
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT):
            if "WRITE THIS CODE - 4 in a row":
                return True
            
def winning_move(board, piece):
    # this one is done, it just uses the functions above
    return  winning_move_horizontal(board, piece) or \
            winning_move_vertical(board, piece) or \
            winning_move_diagonal_up(board, piece) or \
            winning_move_diagonal_down(board, piece)
    

In [ ]:
#
# You also need to finish THIS routine
#
# This routine is the "heart" of the AI intelligence
# It helps evaluate the "quality" of a possible, future board position.
# Argument "window" is an array of exactly 4 board pieces, all next to each other
# They may come from the board horizontally, vertically, or diagonally, it doesn't matter.
# Your job is to "score" how good a board position would be if in included those 4 pieces
# in a row.
#
# The elements in the list contain PLAYER_PIECE (which is 1), AI_PIECE (which is 2), 
# and EMPTY (which is 0)
#   as in:    evaluate_window([1,1,2,0],1)
# The argument "piece" will be either AI_PIECE or PLAYER_PIECE
#
# You might want to adjust these numbers for better results as you play with the program...
#
def evaluate_window(window, piece):
    if piece == PLAYER_PIECE:
        opp_piece = AI_PIECE
    else:
        opp_piece = PLAYER_PIECE
      
    # This is the "score" that the function returns.  It may still be zero at the bottom, or
    # you may have changed it below.
    score = 0


    # If all 4 pieces are equal to "piece", this is the BEST output, 
    # all 4 pieces are the same, so you already won!
    # this should receive a very high positive score (like 100)
    if "WRITE THIS TEST... - something good is found in the array":
        score = 100
        
    # If 3 pieces are equal to "piece", and the other piece is EMPTY, this is also good.
    # This should receive a small, positive 1-digit score
    elif "WRITE THIS TEST... - something OK is found in the array":
        score = 5
        
    # If 2 pieces are equal to "piece", and the other 2 pieces are EMPTY, this is OK.
    # you still have the possibility to win this, but the other player would need
    # to not block you.
    # This should receive a smaller, positive 1-digit score
    elif "WRITE THIS TEST... - something SO-SO is found in the array":
        score = 2

    # The last score to evaluate is if 3 of the pieces belong to the opponent (opp_piece)
    # and the 4th position is empty.  This should receive a NEGATIVE score.  
    # Probably a small one-digit negative number 
    elif "WRITE THIS TEST... - something BAD is found in the array":
        score = -4

    # now, return your analysis to the AI engine
    return score

#
# WARNING - this routine gets called MILLIONS of times and printing debugging code here can 
# take a LONG time and be hard to stop.
# If you want to debug this code, I recommend that you test it in this cell directly by calling it
# like:
#print(evaluate_window([1,1,1,1],1))
#print(evaluate_window([1,0,1,1],1))
#print(evaluate_window([0,0,0,0],1))
#print(evaluate_window([2,2,0,2],1))

## Run this cell to start the game

In [ ]:
play_game()